In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models

In [2]:
import os
import pandas as pd

# Define the folder path where images are stored
image_folder =  "./all_images/"
 # Change this to your actual folder path


In [3]:
# Load the required libraries
import pandas as pd

# Define the file paths
train_file_path = r"train_data_augmented.xlsx"
test_file_path = r"test_data.xlsx"

# Load the datasets
train_df = pd.read_excel(train_file_path,  engine="openpyxl") # For CSV file
test_df = pd.read_excel(test_file_path, engine="openpyxl")  # Specify engine for Excel files

train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

# Display the first few rows to verify the data
train_df.head()

image grade       stage
0  blastocyte_00147.png     B  blastocyte
1       morula00379.png     A      morula
2  blastocyte_00864.png     B  blastocyte
3       morula00166.png     A      morula
4  blastocyte_00299.png     B  blastocyte

In [4]:
train_df.shape

(3204, 3)

In [5]:
test_df.head()

image grade       stage
0  blastocyte_00061.png     A  blastocyte
1  blastocyte_00718.png     B  blastocyte
2      8_cell_00344.png     A      8_cell
3  blastocyte_00351.png     B  blastocyte
4  blastocyte_00746.png     B  blastocyte

In [6]:
import pandas as pd

# Load your dataset (replace with your actual data loading method)
# train_df = pd.read_csv("train.csv")
# test_df = pd.read_csv("test.csv")

# Ensure column names have no leading/trailing spaces
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()

# Print column names to verify
print("Train Data Columns:", train_df.columns)
print("Test Data Columns:", test_df.columns)


print("Processed Data Successfully!")


Train Data Columns: Index(['image', 'grade', 'stage'], dtype='object')
Test Data Columns: Index(['image', 'grade', 'stage'], dtype='object')
Processed Data Successfully!


In [7]:
grade_map = {"A":2, "B":1, "C":0}
stage_map = {"blastocyte":0, "morula":1, "8_cell":2}


In [8]:
train_df.head()

image grade       stage
0  blastocyte_00147.png     B  blastocyte
1       morula00379.png     A      morula
2  blastocyte_00864.png     B  blastocyte
3       morula00166.png     A      morula
4  blastocyte_00299.png     B  blastocyte

In [9]:
train_df.grade = train_df.grade.map(grade_map)
train_df.stage = train_df.stage.map(stage_map)

In [10]:
train_df.head()

image  grade  stage
0  blastocyte_00147.png      1      0
1       morula00379.png      2      1
2  blastocyte_00864.png      1      0
3       morula00166.png      2      1
4  blastocyte_00299.png      1      0

In [11]:
train_df.grade.value_counts()

1    1344
2    1286
0     574
Name: grade, dtype: int64

In [12]:
train_df.stage.value_counts()

1    1363
0     932
2     909
Name: stage, dtype: int64

In [13]:
test_df.grade = test_df.grade.map(grade_map)
test_df.stage = test_df.stage.map(stage_map)

In [14]:
test_df.head()

image  grade  stage
0  blastocyte_00061.png      2      0
1  blastocyte_00718.png      1      0
2      8_cell_00344.png      2      2
3  blastocyte_00351.png      1      0
4  blastocyte_00746.png      1      0

In [15]:
test_df.shape

(419, 3)

In [16]:
# Custom Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['image']
        img_path = os.path.join(self.img_dir, img_name )
        image = Image.open(img_path).convert('RGB')
        
        labels = [int(self.dataframe.iloc[idx][col]) for col in ['grade', 'stage']]
        if self.transform:
            image = self.transform(image)
        return image, tuple(labels)
    
# Define Transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

image_path = "./all_images/"

# Create Dataset and DataLoader
train_dataset = CustomImageDataset(train_df, img_dir=image_path, transform=train_transform)
test_dataset = CustomImageDataset(test_df, img_dir=image_path, transform=test_transform)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## DenseNet

In [16]:
# Def. ne the Model
class MultiLabelDenseNet(nn.Module):
    def __init__(self, num_classes1=3, num_classes2=3):
        super(MultiLabelDenseNet, self).__init__()
        self.base_model = models.densenet121(pretrained=True)
        num_features = self.base_model.classifier.in_features
        self.base_model.classifier = nn.Identity()
        self.fc1 = nn.Linear(num_features, num_classes1)
        self.fc2 = nn.Linear(num_features, num_classes2)

    def forward(self, x):
        features = self.base_model(x)
        return self.fc1(features), self.fc2(features)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiLabelDenseNet().to(device)

# Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


/Users/abinesk/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/abinesk/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:

# Training Function
def train_epoch(model, dataloader, optimizer, device):
    model.train()
    running_loss, total_samples = 0.0, 0
    total_correct1, total_correct2 = 0, 0

    for images, labels in dataloader:
        images = images.to(device)
        label1, label2 = [l.to(device) for l in labels]
        
        optimizer.zero_grad()
        outputs1, outputs2 = model(images)
        loss = sum(criterion(o, l) for o, l in zip([outputs1, outputs2], [label1, label2]))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        total_samples += images.size(0)
        total_correct1 += (outputs1.argmax(1) == label1).sum().item()
        total_correct2 += (outputs2.argmax(1) == label2).sum().item()
    
    return running_loss / total_samples, total_correct1 / total_samples, total_correct2 / total_samples

# Evaluation Function
def eval_epoch(model, dataloader, device):
    model.eval()
    running_loss, total_samples = 0.0, 0
    total_correct1, total_correct2 = 0, 0
    
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            label1, label2 = [l.to(device) for l in labels]
            outputs1, outputs2 = model(images)
            loss = sum(criterion(o, l) for o, l in zip([outputs1, outputs2], [label1, label2]))
            running_loss += loss.item() * images.size(0)
            total_samples += images.size(0)
            total_correct1 += (outputs1.argmax(1) == label1).sum().item()
            total_correct2 += (outputs2.argmax(1) == label2).sum().item()
    
    return running_loss / total_samples, total_correct1 / total_samples, total_correct2 / total_samples

In [ ]:

# Training Loop
num_epochs = 3
for epoch in range(num_epochs):
    train_loss, train_acc1, train_acc2 = train_epoch(model, train_loader, optimizer, device)
    test_loss, test_acc1, test_acc2 = eval_epoch(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f} | Label1 Acc: {train_acc1:.4f} | Label2 Acc: {train_acc2:.4f}")
    print(f"Test Loss: {test_loss:.4f} | Label1 Acc: {test_acc1:.4f} | Label2 Acc: {test_acc2:.4f}")
    
    torch.save(model.state_dict(), f'./trained_model/densenet_epoch_{epoch+1}.pth')

Epoch [1/5]
Train Loss: 0.8451 | Label1 Acc: 0.7578 | Label2 Acc: 0.9064
Test Loss: 0.6048 | Label1 Acc: 0.7936 | Label2 Acc: 0.9535
Epoch [2/5]
Train Loss: 0.5663 | Label1 Acc: 0.8149 | Label2 Acc: 0.9616
Test Loss: 0.6396 | Label1 Acc: 0.8110 | Label2 Acc: 0.9360
Epoch [3/5]
Train Loss: 0.4594 | Label1 Acc: 0.8411 | Label2 Acc: 0.9782
Test Loss: 0.6367 | Label1 Acc: 0.7762 | Label2 Acc: 0.9564


In [12]:


# Example usage
image_path = './all_images/blastocyte_00707.png'
predicted_grade, predicted_stage = inference(image_path, model, test_transform, device)
print(f"Predicted Grade: {predicted_grade}, Predicted Stage: {predicted_stage}")

Predicted Grade: 0, Predicted Stage: 1


In [6]:
ls ./all_images/

00011.png                              blastocyte_00699.png
00012.png                              blastocyte_00700.png
00073.png                              blastocyte_00701.png
00155.png                              blastocyte_00702.png
00173.png                              blastocyte_00703.png
00216.png                              blastocyte_00704.png
00239.png                              blastocyte_00706.png
00240.png                              blastocyte_00707.png
00264.png                              blastocyte_00708.png
00332.png                              blastocyte_00709.png
00349.png                              blastocyte_00710.png
00445.png                              blastocyte_00711.png
00491.png                              blastocyte_00712.png
00499.png                              blastocyte_00713.png
00522.png                              blastocyte_00714.png
00523.png                              blastocyte_00715.png
00525.png                              b

## VIT

In [17]:
# Define the Model using Vision Transformer
class MultiLabelViT(nn.Module):
    def __init__(self, num_classes1=3, num_classes2=3):
        super(MultiLabelViT, self).__init__()
        # Load a pretrained ViT model (ViT-B/16)
        self.base_model = models.vit_b_16(pretrained=True)
        # Obtain the number of features from the original classification head,
        # which is stored in a Sequential container (index 0)
        num_features = self.base_model.heads[0].in_features
        # Remove the original classification head by replacing it with Identity
        self.base_model.heads = nn.Identity()
        # Add three separate fully connected heads for multi-label outputs
        self.fc1 = nn.Linear(num_features, num_classes1)
        self.fc2 = nn.Linear(num_features, num_classes2)

    def forward(self, x):
        # Forward pass through the ViT model to obtain features.
        features = self.base_model(x)
        # If the returned features include a token sequence, extract the class token (first token)
        if features.dim() == 3:
            features = features[:, 0]
        # Pass the features through the three separate heads
        return self.fc1(features), self.fc2(features)

# # Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiLabelViT().to(device)


# Loss, Optimizer remain unchanged
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

/Users/abinesk/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/abinesk/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_acc1, train_acc2 = train_epoch(model, train_loader, optimizer, device)
    test_loss, test_acc1, test_acc2 = eval_epoch(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f} | Label1 Acc: {train_acc1:.4f} | Label2 Acc: {train_acc2:.4f} ")
    print(f"Test Loss: {test_loss:.4f} | Label1 Acc: {test_acc1:.4f} | Label2 Acc: {test_acc2:.4f} ")
    
    torch.save(model.state_dict(), f'./trained_model/vit_epoch_v2_{epoch+1}.pth')

Epoch [1/5]
Train Loss: 1.0553 | Label1 Acc: 0.6910 | Label2 Acc: 0.8605 
Test Loss: 0.7554 | Label1 Acc: 0.7947 | Label2 Acc: 0.9069 
Epoch [2/5]
Train Loss: 0.7310 | Label1 Acc: 0.7622 | Label2 Acc: 0.9363 
Test Loss: 0.9285 | Label1 Acc: 0.7470 | Label2 Acc: 0.8854 
Epoch [3/5]
Train Loss: 0.6093 | Label1 Acc: 0.8031 | Label2 Acc: 0.9494 
Test Loss: 1.3216 | Label1 Acc: 0.7757 | Label2 Acc: 0.7542 
Epoch [4/5]
Train Loss: 0.4904 | Label1 Acc: 0.8311 | Label2 Acc: 0.9722 
Test Loss: 1.1994 | Label1 Acc: 0.7232 | Label2 Acc: 0.8186 
